<a href="https://colab.research.google.com/github/DiegoAnas/SNN-NLP/blob/master/Sentiment%20CNN%20simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow
print(tensorflow.__version__)
import tensorflow.keras as keras

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Dense, Dropout, Activation, concatenate, Input, \
 Embedding, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.datasets import imdb

import numpy as np
from sklearn.model_selection import StratifiedKFold

2.1.0-rc1


In [0]:
# Parameters:
# Word Embedding
max_features = 10000
maxlen = 400
embedding_dims = 250

# Convolution
kernel_size = 3 # or filter length
filters = 250 
pool_size = 4 

# Training
batch_size = 30
epochs = 2

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
x_data = np.concatenate((x_train, x_test))
y_data = np.concatenate((y_train, y_test))
skf = StratifiedKFold(n_splits=5, random_state=42)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)


In [0]:
# Network Architecture
def create_conv_model(activation:str, filter_length, batch_normalize:bool=False,
                         dropout:bool=False, dropout_rate:float=0.1, embedding_dropout:bool=False, 
                         embedding_dropout_rate:float=0.1):
  # Single branch CNN
  var_input = Input(shape=(400,))
  emb_layer = Embedding(max_features,
                      embedding_dims,
                      input_length=maxlen)(var_input)
  if embedding_dropout:
    emb_layer = SpatialDropout1D(rate=embedding_dropout_rate)(emb_layer)
  conv_branches = []
  conv_layer_1 = Conv1D(filters, kernel_size=filter_length, padding='valid', 
                        activation=activation, strides=1)(emb_layer)
  if batch_normalize:
        conv_layer_1 = BatchNormalization()(conv_layer_1)
  maxpooling = GlobalMaxPooling1D()(conv_layer_1)
  if dropout:
      maxpooling = Dropout(rate=dropout_rate, seed=42)(maxpooling)
  dense_layer = Dense(1, activation='sigmoid')(maxpooling)
  model = Model(inputs=var_input, outputs=dense_layer)
  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

In [0]:
#Test w ReLU
for train, test in skf.split(x_data, y_data):
  target = create_conv_model(activation='relu', filter_length=3)
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.3136 - accuracy: 0.8623 - val_loss: 0.2400 - val_accuracy: 0.9032
Epoch 2/2
40000/40000 [==============================] - 43s 1ms/sample - loss: 0.1550 - accuracy: 0.9422 - val_loss: 0.2461 - val_accuracy: 0.9033
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 43s 1ms/sample - loss: 0.3121 - accuracy: 0.8625 - val_loss: 0.2440 - val_accuracy: 0.9012
Epoch 2/2
40000/40000 [==============================] - 43s 1ms/sample - loss: 0.1497 - accuracy: 0.9462 - val_loss: 0.2607 - val_accuracy: 0.8967
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 43s 1ms/sample - loss: 0.3151 - accuracy: 0.8586 - val_loss: 0.2347 - val_accuracy: 0.9050
Epoch 2/2
40000/40000 [==============================] - 42s 1ms/sample - loss: 0.1554 - accuracy: 0.9402 - va

In [0]:
#Test w SeLU
for train, test in skf.split(x_data, y_data):
  target = create_conv_model(activation='selu', filter_length=3)
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 44s 1ms/sample - loss: 0.3140 - accuracy: 0.8627 - val_loss: 0.2393 - val_accuracy: 0.9013
Epoch 2/2
40000/40000 [==============================] - 44s 1ms/sample - loss: 0.1524 - accuracy: 0.9427 - val_loss: 0.2442 - val_accuracy: 0.9051
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 43s 1ms/sample - loss: 0.3091 - accuracy: 0.8626 - val_loss: 0.2603 - val_accuracy: 0.8912
Epoch 2/2
40000/40000 [==============================] - 42s 1ms/sample - loss: 0.1489 - accuracy: 0.9453 - val_loss: 0.2471 - val_accuracy: 0.9024
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 43s 1ms/sample - loss: 0.3130 - accuracy: 0.8612 - val_loss: 0.2360 - val_accuracy: 0.9039
Epoch 2/2
40000/40000 [==============================] - 42s 1ms/sample - loss: 0.1535 - accuracy: 0.9424 - va